In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import unicodedata
import re
import time

# Directorio base del proyecto (asumiendo que este notebook está en /notebooks)
BASE_DIR = Path("..")

# Carpeta donde se guardarán archivos procesados
PROCESSED_DIR = BASE_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

print("BASE_DIR:", BASE_DIR.resolve())
print("PROCESSED_DIR:", PROCESSED_DIR.resolve())

# Lista canónica de variables de suelo que usaremos en todos los libros
VARIABLES_SUELO = [
    "ph_agua_suelo",
    "materia_organica",
    "fosforo_bray_ii",
    "azufre_fosfato_monocalcico",
    "acidez_intercambiable",
    "aluminio_intercambiable",
    "calcio_intercambiable",
    "magnesio_intercambiable",
    "potasio_intercambiable",
    "sodio_intercambiable",
    "cic",  # capacidad de intercambio catiónico
    "conductividad_electrica",
    "hierro_olsen",
    "cobre_disponible",
    "manganeso_olsen",
    "zinc_olsen",
    "boro_disponible",
    "hierro_doble_acido",
    "cobre_disponible_doble_acido",
    "manganeso_doble_acido",
    "zinc_doble_acido",
]

print("Total variables de suelo definidas:", len(VARIABLES_SUELO))
VARIABLES_SUELO


BASE_DIR: D:\ICD-SOIL
PROCESSED_DIR: D:\ICD-SOIL\processed
Total variables de suelo definidas: 21


['ph_agua_suelo',
 'materia_organica',
 'fosforo_bray_ii',
 'azufre_fosfato_monocalcico',
 'acidez_intercambiable',
 'aluminio_intercambiable',
 'calcio_intercambiable',
 'magnesio_intercambiable',
 'potasio_intercambiable',
 'sodio_intercambiable',
 'cic',
 'conductividad_electrica',
 'hierro_olsen',
 'cobre_disponible',
 'manganeso_olsen',
 'zinc_olsen',
 'boro_disponible',
 'hierro_doble_acido',
 'cobre_disponible_doble_acido',
 'manganeso_doble_acido',
 'zinc_doble_acido']

In [2]:
def normalizar_nombre(col: str) -> str:
    """
    Convierte nombres de columnas a:
    - minúsculas
    - sin tildes
    - espacios y caracteres raros → '_'
    """
    col = str(col)

    # Quitar tildes
    col_norm = unicodedata.normalize("NFKD", col)
    col_norm = "".join(c for c in col_norm if not unicodedata.combining(c))

    # Minúsculas
    col_norm = col_norm.lower()

    # Reemplazar todo lo que no sea letra/número por "_"
    col_norm = re.sub(r"[^a-z0-9]+", "_", col_norm)

    # Quitar "_" al inicio/fin
    col_norm = col_norm.strip("_")

    return col_norm


In [3]:
def load_suelos_data(batch_size=50000, pause=0.8):
    """
    Descarga TODOS los datos de:
    https://www.datos.gov.co/resource/ch4u-f3i5.csv

    usando paginación ($limit + $offset).

    Parámetros:
    - batch_size: tamaño del bloque por petición (máx ~50000)
    - pause: pausa (segundos) entre peticiones para no saturar el servidor

    Retorna:
    - DataFrame con todas las filas concatenadas
    """
    base_url = "https://www.datos.gov.co/resource/ch4u-f3i5.csv"
    dfs = []
    offset = 0

    print("📡 Iniciando descarga completa del dataset de suelos...\n")

    while True:
        url = f"{base_url}?$limit={batch_size}&$offset={offset}"
        print(f"   → Descargando filas {offset} a {offset + batch_size} ...")

        df_chunk = pd.read_csv(url, low_memory=False)

        if df_chunk.empty:
            print("\n✅ Descarga completada (no hay más filas).")
            break

        dfs.append(df_chunk)
        offset += batch_size

        # Pausa pequeña para no saturar el servidor
        time.sleep(pause)

    df_final = pd.concat(dfs, ignore_index=True)
    print(f"\n📁 Total descargado: {df_final.shape[0]:,} filas × {df_final.shape[1]} columnas")
    return df_final


In [4]:
raw_cache_path = PROCESSED_DIR / "suelos_raw_full.csv"

if raw_cache_path.exists():
    print("📂 Cargando copia local de suelos_raw_full.csv ...")
    df_raw = pd.read_csv(raw_cache_path, low_memory=False)
    print("✅ Datos cargados desde cache:", df_raw.shape)
else:
    df_raw = load_suelos_data()
    # Guardar copia local para no tener que descargar siempre
    df_raw.to_csv(raw_cache_path, index=False, encoding="utf-8")
    print("💾 Copia local guardada en:", raw_cache_path.resolve())

df_raw.head()


📡 Iniciando descarga completa del dataset de suelos...

   → Descargando filas 0 a 50000 ...
   → Descargando filas 50000 a 100000 ...
   → Descargando filas 100000 a 150000 ...

✅ Descarga completada (no hay más filas).

📁 Total descargado: 92,738 filas × 32 columnas
💾 Copia local guardada en: D:\ICD-SOIL\processed\suelos_raw_full.csv


,secuencial,fecha_de_an_lisis,departamento,municipio,cultivo,estado,tiempo_de_establecimiento,topografia,drenaje,riego,...,conductividad_electrica,hierro_disponible_olsen,cobre_disponible,manganeso_disponible_olsen,zinc_disponible_olsen,boro_disponible,hierro_disponible_doble_acido,cobre_disponible_doble_acido,manganeso_disponible_doble_acido,zinc_disponible_doble_acido
0,1,7/01/2014,NARIÑO,SAN ANDRÉS DE TUMACO,No Indica,No indica,No indica,No indica,No indica,No indica,...,0.133,66.39,1.700,1.700,1.5,0.187,ND,ND,ND,ND
1,2,21/08/2014,HUILA,SANTA MARÍA,Granadilla,Establecido,De 1 a 5 años,Pendiente,Buen drenaje,No Tiene,...,2.749,250.9,3.7,29.4,28.70,2.084,ND,ND,ND,ND
2,3,22/08/2014,ANTIOQUIA,LIBORINA,Café,Por establecer,No indica,Pendiente,Mal drenaje,No Tiene,...,0.328,390,11.10,8.299,5,0.085,ND,ND,ND,ND
3,4,22/08/2014,ANTIOQUIA,LIBORINA,Maracuyá,Por establecer,No indica,Ondulado,Mal drenaje,No Tiene,...,0.171,200,4.2,4.699,1,0.229,ND,ND,ND,ND
4,5,22/08/2014,ANTIOQUIA,LIBORINA,Café,Por establecer,No indica,Ondulado,Mal drenaje,No Tiene,...,0.323,117.9,3.8,4.600,0.700,0.139,ND,ND,ND,ND


In [6]:
print("Número de columnas originales:", len(df_raw.columns))
print(df_raw.columns)

df = df_raw.copy()
df.columns = [normalizar_nombre(c) for c in df.columns]

print("\nNúmero de columnas después de normalizar:", len(df.columns))
df.columns


Número de columnas originales: 32
Index(['secuencial', 'fecha_de_an_lisis', 'departamento', 'municipio',
       'cultivo', 'estado', 'tiempo_de_establecimiento', 'topografia',
       'drenaje', 'riego', 'fertilizantes_aplicados', 'ph_agua_suelo',
       'materia_organica', 'fosforo_bray_ii', 'azufre_fosfato_monocalcico',
       'acidez_kcl', 'aluminio_intercambiable', 'calcio_intercambiable',
       'magnesio_intercambiable', 'potasio_intercambiable',
       'sodio_intercambiable', 'capacidad_de_intercambio_cationico',
       'conductividad_electrica', 'hierro_disponible_olsen',
       'cobre_disponible', 'manganeso_disponible_olsen',
       'zinc_disponible_olsen', 'boro_disponible',
       'hierro_disponible_doble_acido', 'cobre_disponible_doble_acido',
       'manganeso_disponible_doble_acido', 'zinc_disponible_doble_acido'],
      dtype='object')

Número de columnas después de normalizar: 32


Index(['secuencial', 'fecha_de_an_lisis', 'departamento', 'municipio',
       'cultivo', 'estado', 'tiempo_de_establecimiento', 'topografia',
       'drenaje', 'riego', 'fertilizantes_aplicados', 'ph_agua_suelo',
       'materia_organica', 'fosforo_bray_ii', 'azufre_fosfato_monocalcico',
       'acidez_kcl', 'aluminio_intercambiable', 'calcio_intercambiable',
       'magnesio_intercambiable', 'potasio_intercambiable',
       'sodio_intercambiable', 'capacidad_de_intercambio_cationico',
       'conductividad_electrica', 'hierro_disponible_olsen',
       'cobre_disponible', 'manganeso_disponible_olsen',
       'zinc_disponible_olsen', 'boro_disponible',
       'hierro_disponible_doble_acido', 'cobre_disponible_doble_acido',
       'manganeso_disponible_doble_acido', 'zinc_disponible_doble_acido'],
      dtype='object')

In [7]:
print("Columnas normalizadas relevantes (muestra):")
print(df.columns.tolist())

# Intentar detectar columna de fecha de análisis
date_candidates = [
    c for c in df.columns
    if any(k in c for k in ["fecha", "f_analisis", "f_muestreo", "de_an_lisis"])
]

print("\nPosibles columnas de fecha:", date_candidates)

if not date_candidates:
    raise RuntimeError("No se encontró ninguna columna de fecha en el dataset.")

DATE_COL = date_candidates[0]
print("Usando columna de fecha:", DATE_COL)

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL])

print("Rango de fechas:", df[DATE_COL].min(), "→", df[DATE_COL].max())
df[[DATE_COL]].head()


Columnas normalizadas relevantes (muestra):
['secuencial', 'fecha_de_an_lisis', 'departamento', 'municipio', 'cultivo', 'estado', 'tiempo_de_establecimiento', 'topografia', 'drenaje', 'riego', 'fertilizantes_aplicados', 'ph_agua_suelo', 'materia_organica', 'fosforo_bray_ii', 'azufre_fosfato_monocalcico', 'acidez_kcl', 'aluminio_intercambiable', 'calcio_intercambiable', 'magnesio_intercambiable', 'potasio_intercambiable', 'sodio_intercambiable', 'capacidad_de_intercambio_cationico', 'conductividad_electrica', 'hierro_disponible_olsen', 'cobre_disponible', 'manganeso_disponible_olsen', 'zinc_disponible_olsen', 'boro_disponible', 'hierro_disponible_doble_acido', 'cobre_disponible_doble_acido', 'manganeso_disponible_doble_acido', 'zinc_disponible_doble_acido']

Posibles columnas de fecha: ['fecha_de_an_lisis']
Usando columna de fecha: fecha_de_an_lisis
Rango de fechas: 2014-01-04 00:00:00 → 2025-12-08 00:00:00


,fecha_de_an_lisis
0,2014-07-01
163,2014-05-09
164,2014-05-09
165,2014-05-09
166,2014-05-09


In [9]:
# Mapeo de nombres normalizados del portal → nombres canónicos VARIABLES_SUELO
mapping_columnas = {
    "ph_agua_suelo": "ph_agua_suelo",
    "materia_organica": "materia_organica",
    "fosforo_bray_ii": "fosforo_bray_ii",
    "azufre_fosfato_monocalcico": "azufre_fosfato_monocalcico",
    "acidez_kcl": "acidez_intercambiable",  # usamos acidez en KCl como proxy
    "aluminio_intercambiable": "aluminio_intercambiable",
    "calcio_intercambiable": "calcio_intercambiable",
    "magnesio_intercambiable": "magnesio_intercambiable",
    "potasio_intercambiable": "potasio_intercambiable",
    "sodio_intercambiable": "sodio_intercambiable",
    "capacidad_de_intercambio_cationico": "cic",
    "conductividad_electrica": "conductividad_electrica",
    "hierro_disponible_olsen": "hierro_olsen",
    "cobre_disponible": "cobre_disponible",
    "manganeso_disponible_olsen": "manganeso_olsen",
    "zinc_disponible_olsen": "zinc_olsen",
    "boro_disponible": "boro_disponible",
    "hierro_disponible_doble_acido": "hierro_doble_acido",
    "cobre_disponible_doble_acido": "cobre_disponible_doble_acido",
    "manganeso_disponible_doble_acido": "manganeso_doble_acido",
    "zinc_disponible_doble_acido": "zinc_doble_acido",
}

# Crear columnas canónicas a partir de las originales (cuando existan)
for col_src, col_dest in mapping_columnas.items():
    if col_src in df.columns:
        df[col_dest] = df[col_src]
    else:
        # si no existe esa columna en el dataset, creamos NaN
        print(f"⚠️ Columna de origen '{col_src}' no encontrada. '{col_dest}' quedará como NaN.")
        df[col_dest] = np.nan

# Verificar qué VARIABLES_SUELO quedaron realmente presentes (no solo creadas, sino con algo distinto de todo NaN)
variables_presentes = [
    v for v in VARIABLES_SUELO
    if v in df.columns and df[v].notna().any()
]

variables_faltantes = [v for v in VARIABLES_SUELO if v not in variables_presentes]

print("\n✅ Variables de suelo con datos (al menos un valor no nulo):", len(variables_presentes))
print(variables_presentes)

print("\n⚠️ Variables de suelo sin datos (o completamente NaN):", len(variables_faltantes))
print(variables_faltantes)



✅ Variables de suelo con datos (al menos un valor no nulo): 21
['ph_agua_suelo', 'materia_organica', 'fosforo_bray_ii', 'azufre_fosfato_monocalcico', 'acidez_intercambiable', 'aluminio_intercambiable', 'calcio_intercambiable', 'magnesio_intercambiable', 'potasio_intercambiable', 'sodio_intercambiable', 'cic', 'conductividad_electrica', 'hierro_olsen', 'cobre_disponible', 'manganeso_olsen', 'zinc_olsen', 'boro_disponible', 'hierro_doble_acido', 'cobre_disponible_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido']

⚠️ Variables de suelo sin datos (o completamente NaN): 0
[]


In [10]:
# Antes de convertir a numérico, reemplazamos textos típicos como 'ND' por NaN
for col in variables_presentes:
    # Reemplazar valores tipo 'ND', 'nd', 'ND ' etc. por NaN
    df[col] = (
        df[col]
        .replace(["ND", "Nd", "nd", "N/D", "n/d", " NA", "NA", ""], np.nan)
    )

    # Forzar a numérico
    df[col] = pd.to_numeric(df[col], errors="coerce")

df[variables_presentes].dtypes


ph_agua_suelo                   float64
materia_organica                float64
fosforo_bray_ii                 float64
azufre_fosfato_monocalcico      float64
acidez_intercambiable           float64
aluminio_intercambiable         float64
calcio_intercambiable           float64
magnesio_intercambiable         float64
potasio_intercambiable          float64
sodio_intercambiable            float64
cic                             float64
conductividad_electrica         float64
hierro_olsen                    float64
cobre_disponible                float64
manganeso_olsen                 float64
zinc_olsen                      float64
boro_disponible                 float64
hierro_doble_acido              float64
cobre_disponible_doble_acido    float64
manganeso_doble_acido           float64
zinc_doble_acido                float64
dtype: object

In [11]:
# Porcentaje de nulos por variable de suelo
resumen_calidad = (
    df[variables_presentes]
    .isna()
    .mean()
    .to_frame(name="porcentaje_nulos")
)

resumen_calidad["conteo_total"] = len(df)
resumen_calidad["conteo_no_nulos"] = (1 - resumen_calidad["porcentaje_nulos"]) * len(df)

resumen_calidad.sort_values("porcentaje_nulos", ascending=False)


,porcentaje_nulos,conteo_total,conteo_no_nulos
zinc_doble_acido,0.999345,35111,23.0
cobre_disponible_doble_acido,0.963687,35111,1275.0
manganeso_doble_acido,0.952949,35111,1652.0
hierro_doble_acido,0.951639,35111,1698.0
sodio_intercambiable,0.496255,35111,17687.0
acidez_intercambiable,0.494062,35111,17764.0
aluminio_intercambiable,0.494062,35111,17764.0
zinc_olsen,0.417220,35111,20462.0
cobre_disponible,0.312010,35111,24156.0
fosforo_bray_ii,0.174589,35111,28981.0


In [13]:
# Rangos por variable: min y max
resumen_rango = pd.DataFrame({
    "min": df[variables_presentes].min(),
    "max": df[variables_presentes].max(),
})

resumen_rango


,min,max
ph_agua_suelo,3.780,10.14
materia_organica,0.000,68.08
fosforo_bray_ii,0.088,2396.00
azufre_fosfato_monocalcico,-1.010,4862.00
acidez_intercambiable,0.000,26.72
aluminio_intercambiable,0.000,23.03
calcio_intercambiable,0.060,171.80
magnesio_intercambiable,0.011,58.08
potasio_intercambiable,0.090,58.77
sodio_intercambiable,0.002,44.69


In [14]:
# Guardar CSV limpio
output_csv = PROCESSED_DIR / "suelos_clean.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")
print("💾 Dataset limpio (CSV) guardado en:", output_csv.resolve())


💾 Dataset limpio (CSV) guardado en: D:\ICD-SOIL\processed\suelos_clean.csv


In [15]:
# Guardado opcional en Parquet (si tienes pyarrow o fastparquet instalado)
output_parquet = PROCESSED_DIR / "suelos_clean.parquet"

try:
    df.to_parquet(output_parquet, index=False)
    print("🗂️ Dataset limpio (Parquet) guardado en:", output_parquet.resolve())
except Exception as e:
    print("⚠️ No se pudo guardar en Parquet (pyarrow/fastparquet no instalado o error de motor).")
    print("   Detalle:", e)


🗂️ Dataset limpio (Parquet) guardado en: D:\ICD-SOIL\processed\suelos_clean.parquet


In [16]:
df["fecha_de_an_lisis"].min(), df["fecha_de_an_lisis"].max()


(Timestamp('2014-01-04 00:00:00'), Timestamp('2025-12-08 00:00:00'))